#### 1. 패키지 설치

In [ ]:
%pip install -q -U langchain langchain-ollama langgraph

#### 로컬 Ollama로 설치한 deepseek-r1 모델과 ExaOne3 모델을 사용하기
##### ollama run deepseek-r1:7b
##### ollama run exaone3.5

##### 최신버전 LangChain에서는 ChatOllama와 RunnableSequence(prompt | llm) 를 사용

##### deepseeek 모델 9.9 와 9.11 크기 비교문제  (영어로 질문, invoke()함수)

In [5]:
import requests

response = requests.get("http://127.0.0.1:11434")
print(response.text)

Ollama is running


In [ ]:
from langchain_ollama import ChatOllama

try:
    deepseek = ChatOllama(model="deepseek-r1:1.5b", temperature=0.5)    # 모델 호출
    response = deepseek.invoke("which is bigger between 9.9 and 9.11?")
    print(response.content)
except Exception as e:
    print(f"Error: {e}")

##### exaone3.4 모델 9.9 와 9.11 크기 비교문제  (한글로 질문, invoke()함수)

In [ ]:
from langchain_ollama import ChatOllama

try:
    exaone = ChatOllama(model="exaone3.5:2.4b", temperature=0.5, n_gpu_layers=0, batch_size=128)
    #exaone = ChatOllama(model="exaone3.5:2.4b", temperature=0.5)
    # 모델 호출
    response = exaone.invoke("9.9와 9.11 중 무엇이 더 큰가요?")
    print(response.content)
except Exception as e:
    print(f"Error: {e}")

##### deepseeek 모델 9.9 와 9.11 크기 비교문제  (영어로 질문, stream()함수)

In [ ]:
from IPython.display import display, Markdown
from langchain_ollama import ChatOllama

deepseek = ChatOllama(model="deepseek-r1:1.5b", temperature=0.5)

answer = []
for chunk in deepseek.stream("which is bigger between 9.9 and 9.11?"):
    answer.append(chunk.content)
    print(chunk.content, end="", flush=True)

answer_md=''.join([i for i in answer])
display(Markdown(answer_md))

##### deepseeek 모델 9.9 와 9.11 크기 비교문제  (한글로 질문, stream()함수)

In [ ]:
from IPython.display import display, Markdown
from langchain_ollama import ChatOllama

deepseek = ChatOllama(model="deepseek-r1:1.5b", temperature=0.5)

answer = []
for chunk in deepseek.stream("9.9와 9.11 중 무엇이 더 큰가요?"):
    answer.append(chunk.content)
    print(chunk.content, end="", flush=True)

answer_md=''.join([i for i in answer])
display(Markdown(answer_md))

##### Exaone 모델 9.9 와 9.11 크기 비교문제  (한글로 질문, stream()함수)

In [ ]:
from langchain_ollama import ChatOllama

exaone = ChatOllama(model="exaone3.5:2.4b", temperature=0.5)

answer = []
for chunk in exaone.stream("9.9와 9.11 중 무엇이 더 큰가요?"):
    answer.append(chunk.content)
    print(chunk.content, end="", flush=True)

answer_md=''.join([i for i in answer])
display(Markdown(answer_md))


#### DeepSeek의 추론 능력과 ExaOne의 한글 생성 능력 결합하기
* DeepSeek는 태그 안에서 이루어지는 추론을 기반으로 다른 LLM 대비 높은 성능을 발휘합니다.
* 하지만 Ollama에서 제공하는 deepseek r1-distill-qwen 모델은 한국어 생성 능력이 부족합니다.
* DeepSeek의 추론 능력과 ExaOne의 한글 생성 능력 결합해 보겠습니다.

In [ ]:
from langchain_ollama import ChatOllama

reasoning_model = ChatOllama(model="deepseek-r1:1.5b", temperature=0, stop=["</think>"])
reasoning_model                            

In [ ]:
generation_model = ChatOllama(model="exaone3.5:2.4b", temperature=0.7)
generation_model

#### LangGraph 로 연결하기

In [ ]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.prompts import ChatPromptTemplate

#LangGraph에서 State 사용자정의 클래스는 노드 간의 정보를 전달하는 틀입니다. 
#노드 간에 계속 전달하고 싶거나, 그래프 내에서 유지해야 할 정보를 미리 정의힙니다. 
class State(TypedDict):
    question: str
    thinking: str
    answer: str

answer_prompt = ChatPromptTemplate([
    (
        "system",
        """
        당신은 사용자의 질문에 대해 명확하고 포괄적인 답변을 제공하는 AI 어시스턴트입니다.

        당신의 작업은 다음과 같습니다:
        - 질문과 제공된 추론을 신중하게 분석하세요.
        - 추론에서 얻은 통찰력을 포함하여 잘 구조화된 답변을 생성하세요.
        - 답변이 사용자의 질문에 직접적으로 대응하도록 하세요.
        - 정보를 명확하고 자연스럽게 전달하되, 추론 과정을 명시적으로 언급하지 마세요.

        지침:
        - 답변을 대화 형식으로 작성하고, 흥미롭게 전달하세요.
        - 중요한 포인트를 모두 다루면서도 명확하고 간결하게 작성하세요.
        - 제공된 추론을 사용한다는 것을 언급하지 말고, 그 통찰력을 자연스럽게 포함시키세요.
        - 도움이 되고 전문적인 톤을 유지하세요.

        목표: 사용자의 질문에 직접적으로 대응하면서 추론 과정에서 얻은 통찰력을 자연스럽게 포함한 정보 제공입니다.
        """
    ),
    (
        "human",
        """
        질문: {question}
        추론: {thinking}
        """
    )
])
print(answer_prompt)

In [ ]:
#DeepSeek를 통해서 추론 부분까지만 생성합니다. 
def think(state: State):
    question = state["question"]
    response = reasoning_model.invoke(question)
    print(response.content)
    return {"thinking": response.content}

#Exaone를 통해서 결과 출력 부분을 생성합니다.
def generate(state: State):
    messages = answer_prompt.invoke({"question": state["question"], "thinking": state["thinking"]})
    response = generation_model.invoke(messages)
    print(response.content)
    return {"answer": response.content}

# 그래프 컴파일
graph_builder = StateGraph(State).add_sequence([think, generate])
graph_builder.add_edge(START, "think")
graph = graph_builder.compile()

# 입력 데이터
inputs = {"question": "9.9와 9.11 중 무엇이 더 큰가요?"}

# invoke()를 사용하여 그래프 호출
result = graph.invoke(inputs)
print(result)

# 결과 출력
print("생성된 답변:", result["answer"])

In [ ]:
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

display(
    Image(
        graph.get_graph().draw_mermaid_png(draw_method=MermaidDrawMethod.API)        
    )
)

In [3]:
inputs = {"question": "9.9와 9.11 중 무엇이 더 큰가요?"}

async for event in graph.astream_events(inputs, version="v2"):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        print(event['data']['chunk'].content, end="", flush=True)

<think>
First, I need to compare the two numbers: 9.9 and 9.11.

Both numbers have the same whole number part, which is 9.

To make a fair comparison, I'll align their decimal places by writing 9.9 as 9.90.

Now, comparing each digit from left to right:

- The units place for both numbers is 9.
- In the tenths place, 9 has a 9 and 9.11 has a 1.
- Since 9 is greater than 1 in the tenths place, 9.90 is larger than 9.11.

Therefore, 9.9 is greater than 9.11.
추론 결과: <think>
First, I need to compare the two numbers: 9.9 and 9.11.

Both numbers have the same whole number part, which is 9.

To make a fair comparison, I'll align their decimal places by writing 9.9 as 9.90.

Now, comparing each digit from left to right:

- The units place for both numbers is 9.
- In the tenths place, 9 has a 9 and 9.11 has a 1.
- Since 9 is greater than 1 in the tenths place, 9.90 is larger than 9.11.

Therefore, 9.9 is greater than 9.11.

물론입니다! 질문에 대한 답변을 드리겠습니다.

**답변:**

9.9와 9.11을 비교해보면, 두 숫자 모두 정수 부분이 동일한

In [ ]:
from langchain_ollama import ChatOllama
from typing_extensions import TypedDict
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import START, StateGraph

# 모델 설정: 두 개의 서로 다른 모델을 사용하여 추론과 답변 생성을 수행
# - reasoning_model: 추론을 담당하는 모델 (온도 낮음, 정확한 분석용)
# - generation_model: 답변 생성을 담당하는 모델 (온도 높음, 창의적 응답용)
reasoning_model = ChatOllama(model="deepseek-r1:1.5b", temperature=0, stop=["</think>"])
generation_model = ChatOllama(model="exaone3.5:2.4b", temperature=0.7)

# 상태(State) 정의: 그래프에서 상태를 유지하기 위한 데이터 구조
class State(TypedDict):
    question: str   # 사용자의 질문
    thinking: str   # 추론 결과
    answer: str     # 최종 답변

# 답변 생성 프롬프트: AI가 답변을 생성할 때 사용할 지침과 입력 형식 정의
answer_prompt = ChatPromptTemplate([
    (
        "system",
        """
        당신은 사용자의 질문에 대해 명확하고 포괄적인 답변을 제공하는 AI 어시스턴트입니다.

        당신의 작업은 다음과 같습니다:
        - 질문과 제공된 추론을 신중하게 분석하세요.
        - 추론에서 얻은 통찰력을 포함하여 잘 구조화된 답변을 생성하세요.
        - 답변이 사용자의 질문에 직접적으로 대응하도록 하세요.
        - 정보를 명확하고 자연스럽게 전달하되, 추론 과정을 명시적으로 언급하지 마세요.

        지침:
        - 답변을 대화 형식으로 작성하고, 흥미롭게 전달하세요.
        - 중요한 포인트를 모두 다루면서도 명확하고 간결하게 작성하세요.
        - 제공된 추론을 사용한다는 것을 언급하지 말고, 그 통찰력을 자연스럽게 포함시키세요.
        - 도움이 되고 전문적인 톤을 유지하세요.

        목표: 사용자의 질문에 직접적으로 대응하면서 추론 과정에서 얻은 통찰력을 자연스럽게 포함한 정보 제공입니다.
        """
    ),
    (
        "human",
        """
        질문: {question}
        추론: {thinking}
        """
    )
])

# 추론 함수: 사용자의 질문을 기반으로 AI가 논리적 사고를 수행
def think(state: State):
    question = state["question"]                    # 입력받은 질문
    response = reasoning_model.invoke(question)     # 추론 모델을 사용하여 답변 생성
    print("추론 결과:", response.content)            # 추론 결과 출력
    return {"thinking": response.content}           # 추론 결과를 상태로 반환

# 답변 생성 함수: 추론 결과를 바탕으로 AI가 최종적인 답변을 생성
def generate(state: State):
    messages = answer_prompt.invoke({"question": state["question"], "thinking": state["thinking"]})
    response = generation_model.invoke(messages)    # 답변 생성 모델을 사용하여 최종 응답 생성
    print("생성된 답변:", response.content)           # 생성된 답변 출력
    return {"answer": response.content}             # 생성된 답변을 상태로 반환

# 그래프 구성: 상태(State) 간의 흐름을 정의
# - think 함수 -> generate 함수 순서로 실행됨
graph_builder = StateGraph(State).add_sequence([think, generate])

# 그래프의 시작 지점을 설정
graph_builder.add_edge(START, "think")

# 그래프 컴파일: 실행 가능한 상태 머신으로 변환
graph = graph_builder.compile()

# 입력 데이터: AI가 처리할 질문을 정의
inputs = {"question": "9.9와 9.11 중 무엇이 더 큰가요?"}

# 그래프 실행 및 결과 출력
result = graph.invoke(inputs)
print("최종 답변:", result["answer"])

추론 결과: <think>
First, I need to compare the two numbers: 9.9 and 9.11.

Both numbers have the same whole number part, which is 9.

To make a fair comparison, I'll align their decimal places by writing 9.9 as 9.90.

Now, comparing each digit from left to right:

- The units place for both numbers is 9.
- In the tenths place, 9 has a 9 and 9.11 has a 1.
- Since 9 is greater than 1 in the tenths place, 9.90 is larger than 9.11.

Therefore, 9.9 is greater than 9.11.

생성된 답변: 물론입니다! 질문에 대한 답변 드리겠습니다.

**답변:**

9.9와 9.11을 비교해보면, 가장 직관적인 방법은 두 숫자를 동일한 소수점 자릿수로 맞추는 것입니다. 이렇게 하면 다음과 같이 비교할 수 있습니다:

- **9.9**를 **9.90**으로 표현합니다.
- **9.11**은 이미 적절한 형태이지만, 비교를 위해 **9.110**으로 표기해볼 수 있습니다.

두 숫자를 비교해보면:

- **일의 자리**: 두 숫자 모두 9를 가지고 있습니다.
- **십의 자리**: **9.9**의 십의 자리는 9이고, **9.11**의 십의 자리는 1입니다. 따라서 9가 1보다 큽니다.

결론적으로, **9.9**가 **9.11**보다 더 큰 수입니다.
최종 답변: 물론입니다! 질문에 대한 답변 드리겠습니다.

**답변:**

9.9와 9.11을 비교해보면, 가장 직관적인 방법은 두 숫자를 동일한 소수점 자릿수로 맞추는 것입니다. 이렇게 하면 다음과 같이 비교할 수 있습니다:

- **9.9**를 **9.90**으로 표현합니다.

In [ ]:
import gradio as gr
from langchain_ollama import ChatOllama
from typing_extensions import TypedDict
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import START, StateGraph

# 모델 설정: 두 개의 서로 다른 모델을 사용하여 추론과 답변 생성을 수행
# - reasoning_model: 추론을 담당하는 모델 (온도 낮음, 정확한 분석용)
# - generation_model: 답변 생성을 담당하는 모델 (온도 높음, 창의적 응답용)
reasoning_model = ChatOllama(model="deepseek-r1:1.5b", temperature=0, stop=["</think>"])
generation_model = ChatOllama(model="exaone3.5:2.4b", temperature=0.7)

# 상태(State) 정의: 그래프에서 상태를 유지하기 위한 데이터 구조
class State(TypedDict):
    question: str   # 사용자의 질문
    thinking: str   # 추론 결과
    answer: str     # 최종 답변

# 답변 생성 프롬프트: AI가 답변을 생성할 때 사용할 지침과 입력 형식 정의
answer_prompt = ChatPromptTemplate([
    (
        "system",
        """
        당신은 사용자의 질문에 대해 명확하고 포괄적인 답변을 제공하는 AI 어시스턴트입니다.

        당신의 작업은 다음과 같습니다:
        - 질문과 제공된 추론을 신중하게 분석하세요.
        - 추론에서 얻은 통찰력을 포함하여 잘 구조화된 답변을 생성하세요.
        - 답변이 사용자의 질문에 직접적으로 대응하도록 하세요.
        - 정보를 명확하고 자연스럽게 전달하되, 추론 과정을 명시적으로 언급하지 마세요.

        지침:
        - 답변을 대화 형식으로 작성하고, 흥미롭게 전달하세요.
        - 중요한 포인트를 모두 다루면서도 명확하고 간결하게 작성하세요.
        - 제공된 추론을 사용한다는 것을 언급하지 말고, 그 통찰력을 자연스럽게 포함시키세요.
        - 도움이 되고 전문적인 톤을 유지하세요.

        목표: 사용자의 질문에 직접적으로 대응하면서 추론 과정에서 얻은 통찰력을 자연스럽게 포함한 정보 제공입니다.
        """
    ),
    (
        "human",
        """
        질문: {question}
        추론: {thinking}
        """
    )
])

# 추론 함수: 사용자의 질문을 기반으로 AI가 논리적 사고를 수행
def think(state: State):
    question = state["question"]                    # 입력받은 질문
    response = reasoning_model.invoke(question)     # 추론 모델을 사용하여 답변 생성
    return {"thinking": response.content}           # 추론 결과를 상태로 반환

# 답변 생성 함수: 추론 결과를 바탕으로 AI가 최종적인 답변을 생성
def generate(state: State):
    messages = answer_prompt.invoke({"question": state["question"], "thinking": state["thinking"]})
    response = generation_model.invoke(messages)    # 답변 생성 모델을 사용하여 최종 응답 생성
    return {"answer": response.content}             # 생성된 답변을 상태로 반환

# 그래프 생성 함수: 상태(State) 간의 흐름을 정의
def create_graph():
    graph_builder = StateGraph(State).add_sequence([think, generate])
    graph_builder.add_edge(START, "think")
    return graph_builder.compile()

# Gradio 인터페이스 생성 및 실행
def chatbot_interface(message, history):
    graph = create_graph()
    inputs = {"question": message}
    result = graph.invoke(inputs)
    return result["answer"]

iface = gr.ChatInterface(fn=chatbot_interface, title="AI 챗봇", description="질문을 입력하면 AI가 답변을 제공합니다.")

if __name__ == "__main__":
    iface.launch()

c:\Users\vega2\anaconda3\Lib\site-packages\gradio\components\chatbot.py:288: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Traceback (most recent call last):
  File "c:\Users\vega2\anaconda3\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\vega2\anaconda3\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


##### Gradio를 사용하여 챗봇 형태로 작성한 코드

In [2]:
import gradio as gr
from langchain_ollama import ChatOllama
from typing_extensions import TypedDict
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import START, StateGraph

# 모델 설정 함수
def initialize_models():
    reasoning_model = ChatOllama(model="deepseek-r1:1.5b", temperature=0, stop=["</think>"])
    generation_model = ChatOllama(model="exaone3.5:2.4b", temperature=0.7)
    return reasoning_model, generation_model

# 상태(State) 정의
class State(TypedDict):
    question: str  # 사용자의 질문
    thinking: str  # 추론 결과
    answer: str    # 최종 답변

# 답변 생성 프롬프트 설정 함수
def create_answer_prompt():
    return ChatPromptTemplate([
        (
            "system",
            """
            당신은 사용자의 질문에 대해 명확하고 포괄적인 답변을 제공하는 AI 어시스턴트입니다.

            당신의 작업은 다음과 같습니다:
            - 질문과 제공된 추론을 신중하게 분석하세요.
            - 추론에서 얻은 통찰력을 포함하여 잘 구조화된 답변을 생성하세요.
            - 답변이 사용자의 질문에 직접적으로 대응하도록 하세요.
            - 정보를 명확하고 자연스럽게 전달하되, 추론 과정을 명시적으로 언급하지 마세요.

            지침:
            - 답변을 대화 형식으로 작성하고, 흥미롭게 전달하세요.
            - 중요한 포인트를 모두 다루면서도 명확하고 간결하게 작성하세요.
            - 제공된 추론을 사용한다는 것을 언급하지 말고, 그 통찰력을 자연스럽게 포함시키세요.
            - 도움이 되고 전문적인 톤을 유지하세요.

            목표: 사용자의 질문에 직접적으로 대응하면서 추론 과정에서 얻은 통찰력을 자연스럽게 포함한 정보 제공입니다.
            """
        ),
        (
            "human",
            """
            질문: {question}
            추론: {thinking}
            """
        )
    ])

# 추론 함수
def think(state: State, reasoning_model):
    question = state["question"]  # 입력받은 질문
    response = reasoning_model.invoke(question)  # 추론 모델을 사용하여 답변 생성
    return {"thinking": response.content}  # 추론 결과를 상태로 반환

# 답변 생성 함수
def generate(state: State, generation_model, answer_prompt):
    messages = answer_prompt.invoke({"question": state["question"], "thinking": state["thinking"]})
    response = generation_model.invoke(messages)  # 답변 생성 모델을 사용하여 최종 응답 생성
    return {"answer": response.content}  # 생성된 답변을 상태로 반환

# 그래프 생성 함수
# def create_graph():
#     reasoning_model, generation_model = initialize_models()
#     answer_prompt = create_answer_prompt()

#     def think_step(state):
#         return think(state, reasoning_model)

#     def generate_step(state):
#         return generate(state, generation_model, answer_prompt)

#     graph_builder = StateGraph(State).add_sequence([think_step, generate_step])
#     graph_builder.add_edge(START, "think")
#     return graph_builder.compile()

# 그래프 생성 함수
def create_graph():
    reasoning_model, generation_model = initialize_models()
    answer_prompt = create_answer_prompt()

    def think_step(state):
        return think(state, reasoning_model)

    def generate_step(state):
        return generate(state, generation_model, answer_prompt)

    # 그래프 빌더 생성 및 노드 추가
    graph_builder = StateGraph(State)
    graph_builder.add_node("think", think_step)
    graph_builder.add_node("generate", generate_step)
    
    # 엣지 추가
    graph_builder.set_entry_point("think")
    graph_builder.add_edge("think", "generate")
    graph_builder.set_finish_point("generate")
    
    return graph_builder.compile()

# 실행 함수
def chat(question: str):
    graph = create_graph()
    inputs = {"question": question}
    result = graph.invoke(inputs)
    return result["answer"]

# Gradio 인터페이스 설정
def launch_gradio():
    iface = gr.Interface(fn=chat, inputs="text", outputs="text", title="AI 챗봇", description="질문을 입력하면 AI가 답변을 제공합니다.")
    iface.launch()

# 실행 예시
if __name__ == "__main__":
    launch_gradio()


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Traceback (most recent call last):
  File "c:\Users\vega2\anaconda3\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\vega2\anaconda3\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\